<h2>Import Libraries</h2>

In [44]:
import pandas as pd #data preprocessing
import numpy as np
import itertools
import string

#for cross validation and model selection
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score

#base models
##linear and kernel based methods
from sklearn.linear_model import LogisticRegression

##tree-based models 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


##for deep neural networks
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers.legacy import Adam, RMSprop
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics 
from openpyxl import load_workbook
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE  # Importing SMOTE directly instead of RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns

##tensor networks
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#for preprocessing data
from sklearn.preprocessing import OneHotEncoder


##metrics 
#metrics for model evaluation and validation
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef, recall_score, precision_score
from sklearn.metrics import balanced_accuracy_score, auc, roc_curve, precision_recall_curve, log_loss, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import learning_curve
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.calibration import calibration_curve, CalibrationDisplay
from sklearn.metrics import balanced_accuracy_score
from openpyxl import load_workbook
import time
%matplotlib inline
#To show all the rows of pandas dataframe
pd.set_option('display.max_rows', None)

#for hyperparameter optimization
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.model_selection import train_test_split

#model explainabilty
import shap

import os 
import time

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'keras'

In [ ]:
!pip install --upgrade openpyxl

<h2>Load the Data</h2>

In [14]:
# Load the CSV dataset
file_path = r"C:\Users\f5671116\OneDrive - FRG\Downloads\datasets\insurance_claims_sample.csv"
data = pd.read_csv(file_path)

In [15]:
print(data.head())
print(data.columns)

   months_as_customer   age  policy_number policy_bind_date policy_state  \
0                 328  48.0         521585       2014-10-17           OH   
1                 228  42.0         342868       2006-06-27           IN   
2                 134  29.0         687698       2000-09-06           OH   
3                 256  41.0         227811       1990-05-25           IL   
4                 228  44.0         367455       2014-06-06           IL   

  policy_csl  policy_deductable  policy_annual_premium  umbrella_limit  \
0    250/500             1000.0                1406.91               0   
1    250/500             2000.0                1197.22         5000000   
2    100/300             2000.0                1413.14         5000000   
3    250/500             2000.0                1415.74         6000000   
4   500/1000             1000.0                1583.91         6000000   

   insured_zip  ... police_report_available total_claim_amount injury_claim  \
0       466132  ...

In [16]:
print('The training dataset has {} samples and {} features.'.format(data.shape[0], data.shape[1]-1))

The training dataset has 5 samples and 39 features.


In [19]:
# Display the first few rows
data.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48.0,521585,2014-10-17,OH,250/500,1000.0,1406.91,0,466132,...,YES,71610.0,6510.0,13020.0,52080,Saab,92x,2004,Y,NaN
1,228,42.0,342868,2006-06-27,IN,250/500,2000.0,1197.22,5000000,468176,...,?,5070.0,780.0,780.0,3510,Mercedes,E400,2007,Y,NaN
2,134,29.0,687698,2000-09-06,OH,100/300,2000.0,1413.14,5000000,430632,...,NO,34650.0,7700.0,3850.0,23100,Dodge,RAM,2007,N,NaN
3,256,41.0,227811,1990-05-25,IL,250/500,2000.0,1415.74,6000000,608117,...,NO,63400.0,6340.0,6340.0,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44.0,367455,2014-06-06,IL,500/1000,1000.0,1583.91,6000000,610706,...,NO,6500.0,1300.0,650.0,4550,Accura,RSX,2009,N,NaN


<h2>EDA on Insurance Claims Data</h2>

<h2>Data Cleaning</h2>

In [21]:
print(data.head())

   months_as_customer   age  policy_number policy_bind_date policy_state  \
0                 328  48.0         521585       2014-10-17           OH   
1                 228  42.0         342868       2006-06-27           IN   
2                 134  29.0         687698       2000-09-06           OH   
3                 256  41.0         227811       1990-05-25           IL   
4                 228  44.0         367455       2014-06-06           IL   

  policy_csl  policy_deductable  policy_annual_premium  umbrella_limit  \
0    250/500             1000.0                1406.91               0   
1    250/500             2000.0                1197.22         5000000   
2    100/300             2000.0                1413.14         5000000   
3    250/500             2000.0                1415.74         6000000   
4   500/1000             1000.0                1583.91         6000000   

   insured_zip  ... police_report_available total_claim_amount injury_claim  \
0       466132  ...

In [22]:
# Remove the _c39 column
data.drop(columns=['_c39'], inplace=True)

In [23]:
# Fill missing values
# Numerical columns: Use median
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    data[col].fillna(data[col].median(), inplace=True)

In [24]:
# Categorical columns: Use mode
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [25]:
data.isnull().sum()

# Verify no missing values remain
print(data.isnull().sum())

months_as_customer             0
age                            0
policy_number                  0
policy_bind_date               0
policy_state                   0
policy_csl                     0
policy_deductable              0
policy_annual_premium          0
umbrella_limit                 0
insured_zip                    0
insured_sex                    0
insured_education_level        0
insured_occupation             0
insured_hobbies                0
insured_relationship           0
capital-gains                  0
capital-loss                   0
incident_date                  0
incident_type                  0
collision_type                 0
incident_severity              0
authorities_contacted          0
incident_state                 0
incident_city                  0
incident_location              0
incident_hour_of_the_day       0
number_of_vehicles_involved    0
property_damage                0
bodily_injuries                0
witnesses                      0
police_rep

In [26]:
# Encode categorical variables
data = pd.get_dummies(data, drop_first=True)

In [27]:
print(data.columns)

Index(['months_as_customer', 'age', 'policy_number', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip',
       'capital-gains', 'capital-loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim',
       'auto_year', 'policy_bind_date_2000-09-06',
       'policy_bind_date_2006-06-27', 'policy_bind_date_2014-06-06',
       'policy_bind_date_2014-10-17', 'policy_state_IN', 'policy_state_OH',
       'policy_csl_250/500', 'policy_csl_500/1000', 'insured_sex_MALE',
       'insured_education_level_MD', 'insured_education_level_PhD',
       'insured_occupation_craft-repair',
       'insured_occupation_machine-op-inspct', 'insured_occupation_sales',
       'insured_hobbies_reading', 'insured_hobbies_sleeping',
       'insured_relationship_other-relative', 'insured_relationship_own-child',
       'insured_relationship_unmarried', 'incide

In [28]:
# Split the data into features and target
X = data.drop(columns=['fraud_reported_Y'])
y = data['fraud_reported_Y'].apply(lambda x: 1 if x == 'Y' else 0)

In [29]:
print(data.info())
print(data.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 70 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   months_as_customer                      5 non-null      int64  
 1   age                                     5 non-null      float64
 2   policy_number                           5 non-null      int64  
 3   policy_deductable                       5 non-null      float64
 4   policy_annual_premium                   5 non-null      float64
 5   umbrella_limit                          5 non-null      int64  
 6   insured_zip                             5 non-null      int64  
 7   capital-gains                           5 non-null      float64
 8   capital-loss                            5 non-null      int64  
 9   incident_hour_of_the_day                5 non-null      int64  
 10  number_of_vehicles_involved             5 non-null      int64  
 1

In [30]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 70 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   months_as_customer                      5 non-null      int64  
 1   age                                     5 non-null      float64
 2   policy_number                           5 non-null      int64  
 3   policy_deductable                       5 non-null      float64
 4   policy_annual_premium                   5 non-null      float64
 5   umbrella_limit                          5 non-null      int64  
 6   insured_zip                             5 non-null      int64  
 7   capital-gains                           5 non-null      float64
 8   capital-loss                            5 non-null      int64  
 9   incident_hour_of_the_day                5 non-null      int64  
 10  number_of_vehicles_involved             5 non-null      int64  
 1

In [32]:
data.isna().sum().sum()

0

In [33]:
data.head(10)

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,...,police_report_available_YES,auto_make_Chevrolet,auto_make_Dodge,auto_make_Mercedes,auto_make_Saab,auto_model_E400,auto_model_RAM,auto_model_RSX,auto_model_Tahoe,fraud_reported_Y
0,328,48.0,521585,1000.0,1406.91,0,466132,53300.0,0,5,...,1,0,0,0,1,0,0,0,0,1
1,228,42.0,342868,2000.0,1197.22,5000000,468176,0.0,0,8,...,0,0,0,1,0,1,0,0,0,1
2,134,29.0,687698,2000.0,1413.14,5000000,430632,35100.0,0,7,...,0,0,1,0,0,0,1,0,0,0
3,256,41.0,227811,2000.0,1415.74,6000000,608117,48900.0,-62400,5,...,0,1,0,0,0,0,0,0,1,1
4,228,44.0,367455,1000.0,1583.91,6000000,610706,66000.0,-46000,20,...,0,0,0,0,0,0,0,1,0,0


In [34]:
data.describe()

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,...,police_report_available_YES,auto_make_Chevrolet,auto_make_Dodge,auto_make_Mercedes,auto_make_Saab,auto_model_E400,auto_model_RAM,auto_model_RSX,auto_model_Tahoe,fraud_reported_Y
count,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000e+00,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,234.800000,40.800000,429483.40000,1600.000000,1403.384000,4.400000e+06,516752.600000,40660.000000,-21680.000000,9.000000,...,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.600000
std,69.621836,7.120393,178310.81797,547.722558,137.252973,2.509980e+06,85897.590064,25268.221148,30247.512294,6.284903,...,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.547723
min,134.000000,29.000000,227811.00000,1000.000000,1197.220000,0.000000e+00,430632.000000,0.000000,-62400.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,228.000000,41.000000,342868.00000,1000.000000,1406.910000,5.000000e+06,466132.000000,35100.000000,-46000.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,228.000000,42.000000,367455.00000,2000.000000,1413.140000,5.000000e+06,468176.000000,48900.000000,0.000000,7.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,256.000000,44.000000,521585.00000,2000.000000,1415.740000,6.000000e+06,608117.000000,53300.000000,0.000000,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,328.000000,48.000000,687698.00000,2000.000000,1583.910000,6.000000e+06,610706.000000,66000.000000,0.000000,20.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
data.age.describe()

count     5.000000
mean     40.800000
std       7.120393
min      29.000000
25%      41.000000
50%      42.000000
75%      44.000000
max      48.000000
Name: age, dtype: float64

In [36]:
# Display the first few rows of X_train and y_train to verify the split
print(X_train.head())
print(y_train.head())

   months_as_customer   age  policy_number  policy_deductable  \
4                 228  44.0         367455             1000.0   
2                 134  29.0         687698             2000.0   
0                 328  48.0         521585             1000.0   
3                 256  41.0         227811             2000.0   

   policy_annual_premium  umbrella_limit  insured_zip  capital-gains  \
4                1583.91         6000000       610706        66000.0   
2                1413.14         5000000       430632        35100.0   
0                1406.91               0       466132        53300.0   
3                1415.74         6000000       608117        48900.0   

   capital-loss  incident_hour_of_the_day  ...  police_report_available_NO  \
4        -46000                        20  ...                           1   
2             0                         7  ...                           1   
0             0                         5  ...                           0   
3

In [37]:
def convert_datetime_features(data):
    for col in data.columns:
        if np.issubdtype(data[col].dtype, np.datetime64):
            data[col] = data[col].astype('int64') / 10**9  # Convert to seconds since epoch
    return data

# Apply the function to the train and test sets
X_train = convert_datetime_features(X_train)
X_test = convert_datetime_features(X_test)



In [38]:
def check_and_convert_types(data):
    for col in data.columns:
        if data[col].dtype == 'object':
            try:
                data[col] = data[col].astype(float)
            except ValueError:
                print(f"Column {col} cannot be converted to float directly.")
    return data

X_train = check_and_convert_types(X_train)
X_test = check_and_convert_types(X_test)


In [39]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)


In [40]:
# Check class distribution in y_train
print(y_train.value_counts())

0    4
Name: fraud_reported_Y, dtype: int64


<h2>Baseline Model</h2>

In [53]:
# Instantiate the SMOTE oversampler
oversampler = SMOTE(random_state=42)

NameError: name 'SMOTE' is not defined

In [48]:
# Resample the training data to handle imbalance
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Instantiate the Logistic Regression classifier
base_clf = LogisticRegression(max_iter=1000, penalty='l1', solver='saga', class_weight='balanced')

# Fit the model on resampled data
start = time.time()
base_clf.fit(X_train_resampled, y_train_resampled)
end = time.time()

# Predict on the test set
y_pred = base_clf.predict(X_test)

# Calculate balanced accuracy
scores = balanced_accuracy_score(y_test, y_pred)

# Print results
print(f"Balanced Accuracy: {scores:.2f}")
print(f"Time taken: {end - start:.2f}s")

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\Users\f5671116\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py)

In [ ]:
cm = 100*confusion_matrix(y_test, y_pred, normalize= 'true')
# cm = 100*cm.astype('float')/cm.sum()
fig, ax = plt.subplots(figsize = (10,10))
sns.heatmap(cm, annot = True, fmt = '.2f', xticklabels= base_clf.classes_, yticklabels= base_clf.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show(block=False)

print(f"-------------------------{name}---------------------------------")
print("---------------------Classification Report-----------------------")
print(classification_report(y_test, y_pred))
print("-------------------------Ensemble Method------------------------")
print('Balanced Accruacy:', scores)
print("-------------------------Run Time------------------------")
print('Total Run Time:',end)
print("------------------------Confusion Matrix--------------------")

<h2>Hyperameter Tuning using Bayesian Optimisation<h2> 

In [ ]:
#calculate class weights 
y_train_array = np.array(insurance_df['Default'])

#calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes= np.unique(y_train), y = y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}